# 회원 탈퇴를 예측해보자!

In [1]:
# 1개월 전의 이용횟수를 집계한 데이터를 이용하자.

import pandas as pd
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')

In [2]:
customer

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,야간,6000,일반,3.916667,4.0,6,1,1,2019-04-30,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187,HD676663,XXXX,C01,M,2019-03-14,NaN,CA1,0,종일,10500,일반,8.000000,8.0,8,8,0,2019-04-30,1
4188,HD246549,XXXXX,C01,F,2019-03-14,NaN,CA1,0,종일,10500,일반,10.000000,10.0,10,10,0,2019-04-30,1
4189,GD037007,XXXXX,C03,M,2019-03-14,NaN,CA1,0,야간,6000,일반,8.000000,8.0,8,8,0,2019-04-30,1
4190,OA953150,XXXXX,C01,M,2019-03-14,NaN,CA1,0,종일,10500,일반,11.000000,11.0,11,11,0,2019-04-30,1


In [3]:
uselog_months

,연월,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7
...,...,...,...
36837,201903,TS995853,8
36838,201903,TS998593,8
36839,201903,TS999079,3
36840,201903,TS999231,6


In [4]:
year_months = list(uselog_months["연월"].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["연월"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months["연월"]==year_months[i-1]]
    del tmp_before["연월"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

<ipython-input-4-c25a647901a3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.rename(columns={"count":"count_0"}, inplace=True)
<ipython-input-4-c25a647901a3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_before.rename(columns={"count":"count_1"}, inplace=True)
<ipython-input-4-c25a647901a3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.rename(columns={"count":"count_0"}, inplace=True)
<ipython-input-4-c25a647901a3>

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


In [5]:
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer['is_deleted'] == 1]
exit_customer['exit_date'] = None
exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])

for i in range(len(exit_customer)):
  exit_customer['exit_date'].iloc[i] = exit_customer['end_date'].iloc[i] - relativedelta(months=1)
exit_customer['연월'] = pd.to_datetime(exit_customer['exit_date']).dt.strftime('%Y%m')
uselog['연월'] = uselog['연월'].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on = ['customer_id', '연월'], how = 'left')
print(len(uselog))
exit_uselog.head()

<ipython-input-5-6a1f28c6968a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer['exit_date'] = None
<ipython-input-5-6a1f28c6968a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])
<ipython-input-5-6a1f28c6968a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

33851


<ipython-input-5-6a1f28c6968a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer['연월'] = pd.to_datetime(exit_customer['exit_date']).dt.strftime('%Y%m')


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
exit_uselog = exit_uselog.dropna(subset = ['name'])
print(len(exit_uselog))
print(len(exit_uselog['customer_id'].unique()))
exit_uselog.head()

# end_date를 기준으로 탈퇴한 회원들의 exit_date를 알아보는 단계
# end_date가 성립하기 위해서는 그 전 달에 탈퇴 신청을 해야하기 때문에 두 달전까지의 데이터로 탈퇴를 결정할지에 대해 예측을 해줘야한다.

1104
1104


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


In [7]:
# 탈퇴한 데이터 뿐만 아니라 지속회원도 있어야 탈퇴를 할지 안할지 예측할 수 있기 때문에 지속 회원에 관한 데이터도 컬럼으로 추가해줘야한다.

conti_customer = customer.loc[customer['is_deleted'] == 0]
conti_uselog = pd.merge(uselog, conti_customer, on = ['customer_id'], how = 'left')
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset = ['name'])
print(len(conti_uselog))

33851
27422


In [8]:
# 탈퇴한 회원은 1104명인데 지속회원은 월등히 많으므로 class imblance 문제를 불러올 수 있기 때문에 지속 회원을 회원 당 1회만 나오게 중복을 제거해주는 작업을 진행해주어야한다.

In [9]:
conti_uselog = conti_uselog.sample(frac = 1).reset_index(drop = True)
conti_uselog = conti_uselog.drop_duplicates(subset = 'customer_id')
print(len(conti_uselog))
conti_uselog.head()

2842


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201810,GD598089,2,5.0,XXXX,C03,F,2016-08-01,NaN,CA1,...,야간,6000.0,일반,4.583333,4.5,8.0,2.0,1.0,2019-04-30,32.0
1,201809,PL571624,7,7.0,XXXXX,C03,F,2018-01-01,NaN,CA1,...,야간,6000.0,일반,6.833333,7.0,11.0,4.0,1.0,2019-04-30,15.0
2,201805,OA132029,8,5.0,XXXXX,C01,F,2016-06-01,NaN,CA1,...,종일,10500.0,일반,5.000000,5.0,8.0,2.0,1.0,2019-04-30,34.0
3,201812,IK593765,9,10.0,XXXX,C01,F,2018-11-07,NaN,CA1,...,종일,10500.0,일반,8.800000,9.0,10.0,7.0,1.0,2019-04-30,5.0
4,201805,IK059803,6,3.0,XXXXX,C02,F,2015-05-01,NaN,CA1,...,주간,7500.0,일반,4.250000,4.0,6.0,3.0,1.0,2019-04-30,47.0


In [10]:
# 지속 회원과 탈퇴 회원 데이터를 결합해준다.

predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index = True)
print(len(predict_data))
predict_data.head()

3946


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201810,GD598089,2,5.0,XXXX,C03,F,2016-08-01,NaT,CA1,...,6000.0,일반,4.583333,4.5,8.0,2.0,1.0,2019-04-30,32.0,NaN
1,201809,PL571624,7,7.0,XXXXX,C03,F,2018-01-01,NaT,CA1,...,6000.0,일반,6.833333,7.0,11.0,4.0,1.0,2019-04-30,15.0,NaN
2,201805,OA132029,8,5.0,XXXXX,C01,F,2016-06-01,NaT,CA1,...,10500.0,일반,5.000000,5.0,8.0,2.0,1.0,2019-04-30,34.0,NaN
3,201812,IK593765,9,10.0,XXXX,C01,F,2018-11-07,NaT,CA1,...,10500.0,일반,8.800000,9.0,10.0,7.0,1.0,2019-04-30,5.0,NaN
4,201805,IK059803,6,3.0,XXXXX,C02,F,2015-05-01,NaT,CA1,...,7500.0,일반,4.250000,4.0,6.0,3.0,1.0,2019-04-30,47.0,NaN


In [11]:
# 예측할 달의 재적 기간을 만들어주자!
predict_data['period'] = 0
predict_data['now_date'] = pd.to_datetime(predict_data['연월'], format = '%Y%m')
predict_data['start_date'] = pd.to_datetime(predict_data['start_date'])
for i in range(len(predict_data)):
  delta = relativedelta(predict_data['now_date'][i], predict_data['start_date'][i])
  predict_data['period'][i] = int(delta.years*12 + delta.months)
predict_data.head()

<ipython-input-11-ef8d79ae2014>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data['period'][i] = int(delta.years*12 + delta.months)


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201810,GD598089,2,5.0,XXXX,C03,F,2016-08-01,NaT,CA1,...,4.583333,4.5,8.0,2.0,1.0,2019-04-30,32.0,NaN,26,2018-10-01
1,201809,PL571624,7,7.0,XXXXX,C03,F,2018-01-01,NaT,CA1,...,6.833333,7.0,11.0,4.0,1.0,2019-04-30,15.0,NaN,8,2018-09-01
2,201805,OA132029,8,5.0,XXXXX,C01,F,2016-06-01,NaT,CA1,...,5.000000,5.0,8.0,2.0,1.0,2019-04-30,34.0,NaN,23,2018-05-01
3,201812,IK593765,9,10.0,XXXX,C01,F,2018-11-07,NaT,CA1,...,8.800000,9.0,10.0,7.0,1.0,2019-04-30,5.0,NaN,0,2018-12-01
4,201805,IK059803,6,3.0,XXXXX,C02,F,2015-05-01,NaT,CA1,...,4.250000,4.0,6.0,3.0,1.0,2019-04-30,47.0,NaN,36,2018-05-01


In [12]:
# 머신 러닝을 할 데이터 셋을 가공했으니 결측치를 제거해주자.
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1               264
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [13]:
predict_data = predict_data.dropna(subset = ['count_1'])
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2630
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2630
period                  0
now_date                0
dtype: int64

In [14]:
# 결측치 제거를 했으니 필요한 컬럼들만 추출하여 카테고리 변수를 get_dummies을 하자

target_col = ['campaign_name', 'class_name', 'gender', 'count_1', 'routine_flg', 'period', 'is_deleted']
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,일반,야간,F,5.0,1.0,26,0.0
1,일반,야간,F,7.0,1.0,8,0.0
2,일반,종일,F,5.0,1.0,23,0.0
3,일반,종일,F,10.0,1.0,0,0.0
4,일반,주간,F,3.0,1.0,36,0.0


In [15]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_일반,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_야간,class_name_종일,class_name_주간,gender_F,gender_M
0,5.0,1.0,26,0.0,1,0,0,1,0,0,1,0
1,7.0,1.0,8,0.0,1,0,0,1,0,0,1,0
2,5.0,1.0,23,0.0,1,0,0,0,1,0,1,0
3,10.0,1.0,0,0.0,1,0,0,0,1,0,1,0
4,3.0,1.0,36,0.0,1,0,0,0,0,1,1,0


In [16]:
del predict_data['campaign_name_일반']
del predict_data['class_name_야간']
del predict_data['gender_M']
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_종일,class_name_주간,gender_F
0,5.0,1.0,26,0.0,0,0,0,0,1
1,7.0,1.0,8,0.0,0,0,0,0,1
2,5.0,1.0,23,0.0,0,0,1,0,1
3,10.0,1.0,0,0.0,0,0,1,0,1
4,3.0,1.0,36,0.0,0,0,0,1,1


In [17]:
# decisiontree 모델을 사용하여 예측 모델을 만들어보자.
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data['is_deleted'] == 1]
conti = predict_data.loc[predict_data['is_deleted'] == 0].sample(len(exit))

X = pd.concat([exit, conti], ignore_index = True)
y = X['is_deleted']
del X['is_deleted']
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

model = DecisionTreeClassifier(random_state= 0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0.
 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0.
 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1.
 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0.
 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1.
 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1.
 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0.
 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0.
 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0.
 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.
 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1.

In [18]:
results_test = pd.DataFrame({"y_test":y_test, "y_pred":y_test_pred})
results_test.head()

,y_test,y_pred
1589,0.0,0.0
1872,0.0,0.0
858,1.0,1.0
1264,0.0,0.0
1808,0.0,0.0


In [19]:
# 예측 모델을 평가하고 모델을 튜닝해보자.

correct = len(results_test.loc[results_test['y_test'] == results_test['y_pred']])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.876425855513308


In [20]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.876425855513308
0.9828897338403042


In [21]:
# 학습용 데이터의 평가 데이터가 약 98%가 나와 과적합으로 볼 수 있는 상황이다.
# 이에 과적합을 방지하기 위해서는 데이터 늘리기, 변수 재검토, 파라미터 조정 등의 방법을 사용할 수 있다.
# 이번에는 파라미터 조정 방법을 사용해보자.

In [22]:
X = pd.concat([exit, conti], ignore_index = True)
y = X['is_deleted']
del X['is_deleted']
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

model = DecisionTreeClassifier(random_state = 0, max_depth = 5) # 여러 파라미터가 존재하지만 max_depth, 나무 깊이의 최대치를 5로 설정하고 모델을 만들어보았다.
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

# 이전에 비해 성능이 조금 더 좋아졌음을 확인할 수 있었다.

0.9163498098859315
0.9207858048162231


In [23]:
# decisiontree 모델의 꽃인 가장 많은 기여를 하게 되는 변수가 무엇인지 찾아보자

In [24]:
importance = pd.DataFrame({"feature_names":X.columns, "coefficient": model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.340065
1,routine_flg,0.124952
2,period,0.533758
3,campaign_name_입회비무료,0.000000
4,campaign_name_입회비반액할인,0.000000
5,class_name_종일,0.001225
6,class_name_주간,0.000000
7,gender_F,0.000000


In [25]:
# 임의로 데이터를 만들어 모델로 예측해보자

count_1 = 3
routing_flg = 1
period = 10
campaign_name = "입회비무료"
class_name = "종일"
gender = "M"

In [26]:
if campaign_name == "입회비반값할인":
    campaign_name_list = [1, 0]
elif campaign_name == "입회비무료":
    campaign_name_list = [0, 1]
elif campaign_name == "일반":
    campaign_name_list = [0, 0]
if class_name == "종일":
    class_name_list = [1, 0]
elif class_name == "주간":
    class_name_list = [0, 1]
elif class_name == "야간":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
input_data = [count_1, routing_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [27]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0. 1.]]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
